In [15]:
from classiq import *
from classiq.qmod.symbolic import floor
import numpy as np

#nth_to_last(0, list) == list[-1]
def nth_to_last(n:int, list): 
    return list[-1-n]

@qfunc
def tof_CT(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit, target:QBit):
    #apply NOT to target if ctl0==b0, etc
    #keeping semantics as close to original as possible
    #within (change of basis) do CCNOT
    ctls = QArray()
    bind([ctl0,ctl1],ctls)
    within_apply(
        lambda: basischange(b0,b1,ctls[0],ctls[1]),
        lambda: control(ctrl=ctls, stmt_block=lambda: X(target))
    )
    bind(ctls,[ctl0,ctl1])
    return

#Note: this was not originally its own function, but this is very difficult to write inside a lambda.
@qfunc(generative=True)
def basischange(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit):
    if (not b0):
        X(ctl0)
    if (not b1):
        X(ctl1)
    return

@qfunc
def main(ans: Output[QBit]):
    q0 = QBit()
    q1 = QBit()
    allocate(q0)
    allocate(q1)
    allocate(ans)
    tof_CT(False, False, q0, q1, ans)


In [16]:
qmod = create_model(
    main,
    out_file="toftest",
)

qmod_constraint = set_constraints(
    qmod, Constraints(optimization_parameter="depth", max_width=50)
)

qprog = synthesize(qmod)

circuit_width = QuantumProgram.from_qprog(qprog).data.width
circuit_depth = QuantumProgram.from_qprog(qprog).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")
show(qprog)

The circuit width is 3 and the circuit_depth is 12
Opening: https://platform.classiq.io/circuit/2uo0aZeCXwArUfg9VSNHkfzlfas?login=True&version=0.72.1
